# Análisis de regresión
"*Todos los modelos están equivocados, pero algunos son útiles" G.E.P. Box (1919-2013)*

## ¿Qué es el análisis de regresión lineal?

El análisis de regresión lineal es una técnica que se utiliza para pronosticar o explicar el comportamiento de una variable *dependiente* con relación a una o más variables *independientes*. En el área de negocios, la variable dependiente generalmente es la variable que nos interesa explicar o pronosticar (por ejemplo *ventas*), mientras que las variables independientes son aquellas que nos sirven para explicar (por ejemplo *gasto en publicidad*).

Se asume que esta relación se ajusta a una ecuación lineal conocida como ecuación de regresión. Cuando sólo se tiene una variable independiente se conoce como regresión lineal simple: 
$$
y = \beta_0 + \beta_1 x  + \varepsilon
$$
Donde $y$ es la variable dependiente y $x$ es la variable independiente y $\varepsilon$ es el término de error. La interpretación de los resultados nos permitirá:

- Evaluar si la variable independiente tiene un efecto significativo sobre la variable dependiente.
- Evaluar la magnitud del efecto de la variable independiente sobre la dependiente.
- Realizar predicciones con base en las variables independientes.

El procedimiento más común para estimar esta ecuación de regresión es minimizando los errores al cuadrado en un método conocido como *mínimos cuadrados ordinarios*.

Ejemplo: https://phet.colorado.edu/sims/html/least-squares-regression/latest/least-squares-regression_en.html



### Interpretación de un modelo de regresión
En su planteamiento básico la ecuación o modelo de regresión múltiple es la siguiente:
$$
y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 +...+ \beta_k x_k + \varepsilon
$$

Donde:
- $y$ es la variable dependiente.
- $x$ es la variable independiente. Si solo existe una variable independiente se conoce como *regresión lineal simple*, si son dos o más, se llama *regresión lineal múltiple*.
- $\beta_0$ es el término de intersección (constante o intercepto), es el valor que tomaría $y$ si todas las variables fueran igual a cero (esto solo tiene sentido en algunos casos). 
- $\beta_1$ es el coeficiente de regresión para cada $x$. Es el cambio estimado en la variable dependiente por unidad de cambio en la variable independiente, con todo lo demás constante.
- $\varepsilon$ es el término de error. Cuando corresponde al resultado de un modelo en específico es preferible referirse al error como *residual*

### Supuestos básicos del modelo
Las estimaciones de los coeficientes de regresión serán óptimas, de acuerdo al teorema de Gauss-Markov, si cumplen los siguientes supuestos con respecto al término de error:
- El error sigue una distribución normal.
- El error tenga media igual a cero.
- Los errores son independientes (no están relacionados entre sí).
- La varianza del error es constante.
Esto se resume como que e~N<sub>iid</sub> (0,σ<sup>2</sup>). Si se cumplen estos supuestos, los estimadores serán los mejores estimadores insesgados de mínima varianza (*MELI* por las siglas en español; *BLUE* por las siglas en inglés). 

## Análisis de regresión
Utilizaremos un modelo de regresión para estimar el precio de una casa en una zona de la ciudad. Como primer paso importamos las bibliotecas a utilizar. En esta ocasión agregaremos la biblioteca warnings para evitar notificaciones de cambios futuros

In [ ]:
# Importar bibliotecas
#import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Características de la muestra
Existen varias recomendaciones para determinar qué *tamaño de muestra* es adecuado. Para generalizar los resultados, lo mínimo recomendable son 5 observaciones por cada variable independiente, aunque lo deseable es tener de 15 a 20 observaciones por cada variable independiente  (Hair, Black, Babin, & Anderson, 2009).  
Green (1991) considera inadecuado establecer una constante (p. ej. 100 observaciones) pero sugiere que para evaluar un modelo se deben tener 50+8*k* observaciones, donde *k* es el número de variables independientes. Si se desea evaluar los coeficientes de regresión de cada variable recomienda 104+*k* observaciones.  
Finalmente, además del tamaño de la muestra, es importante asegurarse de que la muestra sea *representativa* de la población.

In [ ]:
# Importar el archivo "data/b01_casas.xlsx"
df = pd.read_excel('data/b01_casas.xlsx')
# Para este ejemplo seleccionamos casas (tipo = 0 )
df = df[df['tipo']==0] 

In [ ]:
# Revisar las variables y el número de observaciones
df.info()

In [ ]:
# Revisar las primeras filas del DataFrame
df.head(3)

### Características de las variables
En el modelo de regresión lineal múltiple, la variable dependiente debe ser cuantitativa (con escala de medición de intervalo o de razón) y al menos una variable independiente también deberá ser cuantitativa.  

In [ ]:
# Identifica las variables cuantitativasy obtener la estadística descriptiva
var_cont = df[['preciomillones', 'recamaras', 'baños', 'construccion']]
var_cont.describe().T

Las variables independientes no deben presentar *multicolinealidad*, esto es, que una combinación lineal de dos o más variables contengan la misma información que otra variable independiente. Aunque hay análisis formales para ello, un primer acercamiento es tratar de evitar que existan altas correlaciones (r > 0.90) entre dos variables independientes.

In [ ]:
# Obtenemos la matriz de correlaciones
var_cont.corr()

La función `sns.pairplot`es útil para visualizar las relaciones entre pares de variables en un DataFrame. El primer parámetro indica el nombre del DataFrame, *corner* sirve para mostrar solo la matriz triangular inferior, *kind* permite indicar el tipo de gráfico (observa cómo cambia con 'reg' en vez de 'scatter'), *makers* se utiliza para indicar el tipo de marcas en el gráfico (observa cómo cambia con '+' en vez de '.'), y *height*  es la altura en pulgadas de cada gráfico.

In [ ]:
# Graficamos las relaciones entre variables
sns.pairplot(var_cont, corner=True, kind='reg', markers='+', height=1.5);

### Estimación del modelo (con StatsModels)
Utilizaremos primero la biblioteca *StatsModels* para realizar el análisis de regresión

In [ ]:
# Importar la librería
import statsmodels.api as sm

Es recomendable crear un dataframe 'X' con las variables independientes y otro "y" con la variable dependiente. Para seleccionar las variables independientes primero evalúa cuáles pueden ser relevantes. Cuando se tienen muchas variables independientes se puede realizar un **Análisis Factorial** para reducir la dimensionalidad. Para este ejemplo, empezaremos con la variable "construcción". La opción de StatsModel que utilizaremos, requiere crear una columna de "1" para representar la constante en el modelo de regresión.

In [ ]:
# Creamos un DataFrame con las v. indep. y la v. dependiente
X = df[['construccion', 'baños']]
X = sm.add_constant(X)
y = df['preciomillones']

Corremos el modelo de regresión mediante mínimos cuadrados ordinarios:  
`regresion = sm.OLS(y, X).fit()`

In [ ]:
regresion = sm.OLS(y, X).fit()

El resultado se muestra con la instrucción
`summary()`.

In [ ]:
regresion.summary()

In [ ]:
# Coeficientes de regresión
regresion.params

In [ ]:
# Residuales
residuales = regresion.resid

In [ ]:
# Valores estimados
y_hat = regresion.predict(X)

In [ ]:
# Pronóstico
nuevos_valores = [1,500,2]
regresion.predict(nuevos_valores)

## Interpretación de los resultados
- R-squared: entre más cercano a 1 es mejor
- Adj. R-squared: se utiliza para comparar modelos con diferente número de variables independientes, debido a que la R cuadrada ajustada penaliza el incluir variables no significativas.
- AIC (Criterio de información de Akaike)/ BIC (Criterio de información Bayesiano): también son medidas relativas que se utilizan para comparar modelos de regresión múltiple e identificar el de mejor ajuste y simplicidad (menos es mejor).
- F-statistic: evalúa la significancia del modelo en conjunto, la hipótesis nula es que todos los coeficientes de regresión son iguales a cero.
- Skew: o *sesgo* es una medida de la asimetría de la distribución
- Kurtosis: indica qué tan "puntiaguda" o "achatada" es una distribución.
- Prob(Omnibus): es una prueba para evaluar la normalidad en los residuales.
- Jarque-Bera(JB) es un estadístico que sirve para evaluar la normalidad en los residuales.
- Durbin-Watson: sirve para evaluar la presencia de autocorrelación. Valores entre 0 y 2 indican una autocorrelación positiva y valores entre 2 y 4 indican una autocorrelación negativa. Valores cercanos a 2 son deseables.
- Cond. No. es el número de condición, que sirve para evalur problemas de multicolinealidad. Valores arriba de 30 se consideran problemáticos.

### Ajuste del modelo
La métrica más utilizada para medir el ajuste del modelo es el *coeficiente de determinación* (R<sup>2</sup>) que mide la proporción de la variación de la variable dependiente que es explicada por el modelo. En un modelo de regresión lineal, la variación total de los datos  se mide con sumas de cuadrados:
$$
SS_{total} = SS_{modelo} + SS_{residual}
$$
Donde:

$SS_{total} = \sum_{i = 1}^{n} (y-\bar{y})^2$ es la suma de cuadrados total  
$SS_{modelo} = \sum_{i = 1}^{n} (\hat{y}-\bar{y})^2$ es la suma de cuadrados del modelo también conocida como suma de cuadrados explicada  
$SS_{residual} = \sum_{i = 1}^{n} (y-\hat{y})^2$ es la suma de cuadrados del residual o suma de cuadrados del error

El coeficiente de determinación es equivalente a

$$
R^2 = SS_{modelo} / SS_{total}
$$

La R cuadrada toma valores entre 0 y 1, donde valores más grandes indican un mejor ajuste. Sin embargo, no existen criterios únicos de qué valor de R cuadrada es aceptable, sino que ello depende del área de estudio

## Evaluación de supuestos
### Homocedasticidad
Uno de los supuestos del modelo de regresión es que la varianza de los residuales se mantiene constante (homocedasticidad). El cumplimiento de este supuesto se puede explorar en la gráfica del residual estandarizado versus $\hat{Y}$. Por ejemplo, un patrón de dispersión creciente (o decreciente) indicaría una violación a este supuesto.

Existen varias pruebas que se pueden realizar para evaluar la presencia de heteroscedasticidad, tales como:
- Breusch-Pagan: calcula la relación entre los residuales al cuadrado y las variables independientes, si la relación es significativa indica heteroscedasticidad..
- Prueba de White: similar a la Breush-Pagan, considera la correlación serial.
- Prueba de Goldfeld-Quandt: divide los datos con base en los valores de una variable independiente y compara las varianzas de los residuales en ambos grupos.

In [ ]:
# Gráfico de residuales
plt.figure(figsize=(6, 3))
plt.scatter(y_hat, residuales, marker='.', color='b')
plt.xlabel('y hat')
plt.ylabel('residuales');

In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan

# Asumiendo que ya se tiene el modelo y los residuales
lm, lm_p_value, fvalue, f_p_value = het_breuschpagan(residuales, X)

print("Estadístico LM:", lm)
print("Valor p del estadístico LM:", lm_p_value)
print("Estadístico F:", fvalue)
print("Valor p del estadístico F:", f_p_value)

Si el p-valor es menor que 0.05 es evidencia de la presencia de heteroscedasticidad

*¿Qué hacer si se detecta heteroscedasticidad?*
- Explorar transformaciones de variables (p. ej. logaritmos)
- Utilizar otros modelos de regresión, como Mínimos Cuadrados Ponderados (WLS) que asigna pesos inversamente proporcionales a la varianza de los errores
- Crear modelos separados.

### Normalidad
La normalidad en la distribución de los errores y su media se puede verificar en el histograma de los residuales y una prueba de normalidad (como Shapiro-Wilks o  Kolmogorov-Smirnov). En la prueba de normalidad lo deseable es obtener un p-valor mayor a 0.05 (no se rechaza la hipótesis nula de normalidad en los datos). 

El reporte de salida de StatsModelo muestra el p-valor de las pruebas omnibus y de Jarque-Bera, ambas basadas en el sesgo y la curtosis.

In [ ]:
plt.figure(figsize=(4, 3)) 
sns.histplot(residuales, color='b');

In [ ]:
from scipy.stats import shapiro
stat, p_value = shapiro(residuales)
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula de normalidad en los datos")
else:
    print("No se puede rechazar la hipótesis nula de normalidad en los datos")

*¿Qué hacer si se detecta no normalidad en los residuales?*

- Revisar outliers o valores influyentes.
- Explorar transformaciones de variables (p. ej. logaritmos)
- Utilizar otros modelos de regresión.
- Explorar variables no incluidas en el modelo de regresión.
- Utilizar bootstrapping para las estimaciones de intervalos.

### Independencia en los errores
Los errores deben ser independientes en una regresión, esto es, no relacionados entre sí.  Generalmente esto implica evaluar la autocorrelación (correlación entre valores adyacentes) y se lleva a cabo mediante la prueba de Durbin-Watson (solamente en series de tiempo). En esta prueba, el estadístico de prueba puede tomar valores entre 0 y 4. Si el estadístico es cercano a 2 entonces los residuales no están correlacionados. Valores menores a 1 o mayores a 3 son señal de un problema serio de autocorrelación. 

*¿Qué hacer si se detecta autocorrelación en los residuales?*
- Explorar modelos específicos para series temporales (ARIMA, SARIMA)

### Multicolinealidad
La multicolinealidad se presenta cuando existe una fuerte correlación entre dos o más variables independientes o una combinación lineal de ellas. La multicolinealidad puede llevar a tener un modelo estadísticamente significativo, pero coeficientes de regresión no significativos.

Para evaluar la multicolinealidad:
- Revisar la matriz de correlaciones de las variables independientes para identificar correlaciones arriba de 0.90
- Revisar el número de condición, valores mayores a 30 indicarían un problema grave
- Calcular el Factor de Inflación de Varianza de cada variable, valores mayores a 10 indicarían un problema grave

*¿Qué hacer si se detecta multicolinealidad?*
- Regularizar (regresión Lasso o Ridge).
- Combinar variables.
- Eliminar variables redundantes.

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = df[['construccion', 'baños']]
X = sm.add_constant(X)

# Calcular valores VIF para cada variable
vif_data = pd.DataFrame()
vif_data['Variable'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

vif_data

## Extra: uso de scikit-learn

In [ ]:
# Importar la biblioteca
from sklearn.linear_model import LinearRegression

Con *SciKit-Learn' no se agrega la columna de constantes como en StatsModel

In [ ]:
X = df[['construccion']]
y = df['preciomillones']
model = LinearRegression().fit(X, y)

In [ ]:
# Obtener los coeficientes de regresión
model.intercept_, model.coef_

In [ ]:
# Obtener y estimada (y hat)
y_hat = model.predict(X)

In [ ]:
# Obtener residuales
residuales = (y - y_hat).values

In [ ]:
# Calcular R cuadrada
from sklearn.metrics import r2_score
r2_score(y, y_hat)